In [44]:
import pandas as pd

path = '/home/local/work/data/hgsvc/figSX_panels/assm_completeness/cytobands.h5'
df = pd.read_hdf(path, 'cache')
df = df.xs('contig', level='input_type')
df = df.xs('HAP', level='assembly')

stat_types = ['fraction'] #, 'accessible']

hetchrom_file = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38/GRCh38_cytobands_HET.bed'

het_bands = []
het_length = 0
with open(hetchrom_file, 'r') as bed:
    for line in bed:
        chrom, start, end, name, _ = line.strip().split()
        het_length += (int(end) - int(start))
        het_bands.append(line.split()[3])
print(het_length)

euchrom_file = '/home/local/work/code/github/project-diploid-assembly/annotation/grch38/GRCh38_cytobands_EUC.bed'
euc_length = 0
with open(euchrom_file, 'r') as bed:
    for line in bed:
        if line.startswith('#'):
            continue
        chrom, start, end, name, _ = line.strip().split()
        if chrom == 'chrY':
            continue
        euc_length += (int(end) - int(start))

print(euc_length)
print(euc_length + het_length)
        
df['is_het'] = False
df.loc[df['name'].isin(het_bands), 'is_het'] = True
    
out_table = []

for st in stat_types:
    print('### Statistic: ', st)
    subset = df.xs(st, level='stat_type')
    select = subset.groupby(['sample', 'platform', 'hap'])['coverage'].mean()
    select = select.round(2)
    select.rename('cov_full_{}'.format(st), inplace=True)
    out_table.append(select)
    
    subset = subset.loc[~subset['is_het'], :].copy()
    select = subset.groupby(['sample', 'platform', 'hap'])['coverage'].mean()
    select = select.round(2)
    select.rename('cov_euchrom_{}'.format(st), inplace=True)
    out_table.append(select)


new_df = pd.concat(out_table, axis=1, ignore_index=False)
print('### ALL ###')
print(new_df['cov_full_fraction'].describe())
print(new_df['cov_euchrom_fraction'].describe())
print('### HIFI ###')
hifi_df = new_df.xs('HiFi', level='platform')
print(hifi_df['cov_full_fraction'].describe())
print(hifi_df['cov_euchrom_fraction'].describe())
print('### CLR ###')
clr_df = new_df.xs('CLR', level='platform')
print(clr_df['cov_full_fraction'].describe())
print(clr_df['cov_euchrom_fraction'].describe())

dump = '/home/local/work/data/hgsvc/figSX_panels/assm_completeness/hap_cov_cytobands.tsv'
new_df.sort_index(axis=0, level=['platform', 'sample', 'hap'], inplace=True)

new_df.to_csv(dump, sep='\t', index=True, header=True)

495600000
2535442417
3031042417
### Statistic:  fraction
### ALL ###
count    88.000000
mean     91.787727
std       0.452461
min      88.730000
25%      91.637500
50%      91.830000
75%      91.972500
max      92.440000
Name: cov_full_fraction, dtype: float64
count    88.000000
mean     90.980000
std       0.475276
min      87.850000
25%      90.827500
50%      91.020000
75%      91.195000
max      91.670000
Name: cov_euchrom_fraction, dtype: float64
### HIFI ###
count    28.000000
mean     92.099643
std       0.208353
min      91.580000
25%      91.970000
50%      92.085000
75%      92.250000
max      92.440000
Name: cov_full_fraction, dtype: float64
count    28.000000
mean     91.311429
std       0.223071
min      90.730000
25%      91.205000
50%      91.295000
75%      91.462500
max      91.670000
Name: cov_euchrom_fraction, dtype: float64
### CLR ###
count    60.000000
mean     91.642167
std       0.462917
min      88.730000
25%      91.617500
50%      91.750000
75%      91.850000